In [27]:
import os
import openai

In [28]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [29]:
!pip install Gitpython

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [30]:
from git import Repo

In [31]:
from pathlib import Path

In [32]:
#Just testing printing working directory

In [33]:
pwd

'C:\\Users\\david\\Desktop\\Python\\Courses\\Udemy\\OpenAI Python Bootcamp\\dawidahs.github.io'

In [34]:
PATH_TO_BLOG_REPO = Path('C:\\Users\\david\\Desktop\\Python\\Courses\\Udemy\\OpenAI Python Bootcamp\\dawidahs.github.io\\.git')

In [35]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [36]:
PATH_TO_CONTENT = PATH_TO_BLOG/'content'

In [37]:
PATH_TO_CONTENT

WindowsPath('C:/Users/david/Desktop/Python/Courses/Udemy/OpenAI Python Bootcamp/dawidahs.github.io/content')

In [38]:
PATH_TO_CONTENT.mkdir(exist_ok=True,parents=True)

In [39]:
def update_blog(commit_message="Updates blog"):
    repo = Repo(PATH_TO_BLOG_REPO)
    repo.git.add(all=True)
    repo.index.commit(commit_message)
    origin = repo.remote(name='origin')
    origin.push()

In [40]:
random_text_string = 'smlkmssdlmsdl'

In [41]:
with open(PATH_TO_BLOG/'index.html', 'w') as f:
    f.write(random_text_string)

In [42]:
update_blog()